In [1]:
from utils.data_preprocess import *
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
df = pd.read_csv('ETHBTC-5m-data.csv')

In [3]:
df = data_split(df, 3)[0]

In [4]:
data = df.set_index('Timestamp').iloc[:, :4]

In [5]:
train_df, val_df, test_df = train_val_test_split(data)

In [15]:
len(train_df), len(val_df), len(test_df)

(105836, 30239, 15120)

In [12]:
test = DataPreprocess(train_df, val_df, test_df, window_size=24,
                      label_columns=['Close'], label_size=1)

In [13]:
test

Total window size: 24
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24]
Label column name(s): ['Close']

In [14]:
for input, label in test.train.take(1):
    print(f'Inputs shape (batch, time, features): {input.shape}')
    print(f'Labels shape (batch, time, features): {label.shape}')

Inputs shape (batch, time, features): (32, 24, 4)
Labels shape (batch, time, features): (32, 1, 1)
